In [1]:
!pip install --upgrade mlxtend

     |████████████████████████████████| 1.3 MB 31.1 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [2]:
from IPython.display import Image

# 14. 텐서플로 구조
* 텐서플로 v2 그래프
* 그래프 컴파일을 위한 함수 데코레이션
* 텐서플로 변수 사용
* 고전적인 XOR 문제로 모델 수용 능력 이해
* 케라스의 Model 클래스와 함수형 API를 사용한 복잡한 신경만 모델
* 자동 미분과 tf.GradientTape를 사용한 그레이디언트
* tf.estimator

# 14.2 텐서플로의 계산 그래프
* 텐서플로 v2의 즉시 실행과 동적 계산 그래프로 마이그레이션

### 14.2.1 계산 그래프 이해
* 텐서플로는 계산 그래프에 크게 의존한다
* 계산 그래프를 사용하여 입력에서 출력까지 텐서 간의 관계를 유도

In [3]:
# 랭크 0(스칼라)인 텐서 a, b, c를 사용하여 z = 2*(a-b)+c
Image(url='https://git.io/JL52v', width=500)

### 14.2.2 텐서플로 v1.x에서의 그래프
* 프로세스
    1. 새로운 빈 계산 그래프를 만든다
    2. 이 계산 그래프에 노드(텐서나 연산)를 추가
    3. 그래프를 평가(실행)한다
        1. 새로운 세션을 시작한다
        2. 그래프 내 변수를 초기화한다.
        3. 이 세션에서 계산 그래프를 실행한다.

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# z = 2 * (a-b) + c 그래프
g = tf.Graph() # 그래프 정의
with g.as_default(): # 그래프 g에 노드 추가
    a = tf.constant(1, name='a')
    b = tf.constant(2, name='b')
    c = tf.constant(3, name='c')
    z = 2 * (a - b) + c

with tf.compat.v1.Session(graph=g) as sess: # 그래프를 실행하여 텐서 z 평가
    print('결과: z=', sess.run(z))
    print('결과: z=', z.eval())

결과: z= 1
결과: z= 1


### 14.2.3 텐서플로 v2로 이전
* 동적 그래프(즉시 실행)
    * 바로바로 연산 평가 가능
    * 그래프와 세션을 명시적으로 만들 필요가 없고 개발 작업 흐름이 훨씬 간편

In [8]:
a = tf.constant(1, name='a')
b = tf.constant(2, name='b')
c = tf.constant(3, name='c')
z = 2 * (a-b) + c
tf.print('결과: z=', z)

결과: z= 1


### 14.2.4 입력 데이터를 모델에 주입: 텐서플로 v1.x
* 입력 데이터를 모델에 전달하기 위해 플레이스홀더 변수를 만들어야 한다.

In [9]:
g = tf.Graph()
with g.as_default():
    a = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_a')
    b = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_b')
    c = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_c')
    z = 2*(a - b) + c

with tf.compat.v1.Session(graph=g) as sess:
    feed_dict = {a:1, b:2, c:3}
    print('결과: z =', sess.run(z, feed_dict=feed_dict))

결과: z = 1


### 14.2.5 입력 데이터를 모델에 주입: 텐서플로 v2
* 파이썬 변수나 넘파이 배열을 모델에 데이터로 바로 주입 가능

In [10]:
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

tf.print('스칼라 입력:', compute_z(1, 2, 3))
tf.print('랭크 1 입력:', compute_z([1], [2], [3]))
tf.print('랭크 2 입력:', compute_z([[1]], [[2]], [[3]]))

스칼라 입력: 1
랭크 1 입력: [1]
랭크 2 입력: [[1]]


### 14.2.6 함수 데코레이터로 계산 성능 높이기
* 파이썬 코드를 텐서플로 그래프 코드로 자동 변환해주는 AutoGraph 도구를 제공
* 계산 효율을 위한 보통의 파이썬 함수를 텐서플로의 정적 그래프로 컴파일 방법

In [11]:
@tf.function # 동적 타이핑 처리
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

tf.print('스칼라 입력:', compute_z(1, 2, 3))
tf.print('랭크 1 입력:', compute_z([1], [2], [3]))
tf.print('랭크 2 입력:', compute_z([[1]], [[2]], [[3]]))

스칼라 입력: 1
랭크 1 입력: [1]
랭크 2 입력: [[1]]


* 입력 매개변수를 기반으로 그래프를 구성하기 위해 트레이싱 기법 사용
    * 트레이싱 기법을 사용해 함수 호출에 사용된 입력 시그니처를 기반으로 키의 튜플을 생성
    * 키 생성
        * tf.Tensor 매개변수이면 키는 크기와 데이터 타입을 기반으로 한다.
        * 리스트와 같은 파이썬 타입이면 id() 함수를 사용하여 키를 생성.
        * 파이썬 원시 자료형이면 키는 입력 값을 기반으로 한다.


In [12]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),
                              tf.TensorSpec(shape=[None], dtype=tf.int32),
                              tf.TensorSpec(shape=[None], dtype=tf.int32),))

def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

# 랭크1 텐서(또는 랭크 1 텐서로 바꿀 수 있는 리스트)를 사용하여 함수 호출 가능
tf.print('랭크 1 입력:', compute_z([1], [2], [3]))
tf.print('랭크 1 입력:', compute_z([1, 2], [2, 4], [3, 6]))

랭크 1 입력: [1]
랭크 1 입력: [1 2]


* 1이 아닌 랭크를 가진 텐서를 사용하여 호출하면 지정된 입력 시그니처와 맞지 않아 에러 발생
        tf.print('Rank 0 Inputs:', compute_z(1, 2, 3))
        tf.print('Rank 2 Inputs:', compute_z([[1], [2]], [[2], [4]], [[3], [6]]))

# 14.3 모델 파라미터를 저장하고 업데이트하기 위한 텐서플로 변수 객체
* Variable: 훈련하는 동안 모델 파라미터를 저장하고 업데이트할 수 있는 특별한 Tensor 객체.

In [16]:
a = tf.Variable(initial_value=3.14, name='var_a')
b = tf.Variable(initial_value=[1, 2, 3], name='var_b')
c = tf.Variable(initial_value=[True, False], dtype=tf.bool)
d = tf.Variable(initial_value=['abc'], dtype=tf.string)
print(a)
print(b)
print(c)
print(d)

<tf.Variable 'var_a:0' shape=() dtype=float32, numpy=3.14>
<tf.Variable 'var_b:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=bool, numpy=array([ True, False])>
<tf.Variable 'Variable:0' shape=(1,) dtype=string, numpy=array([b'abc'], dtype=object)>


Variable을 만들 때 항상 초기값을 제공해야 한다. 변수는 trainable 속성을 가지며 기본값은 True이다. 케라스 같은 API는 이 속성을 사용해서 훈련하는 변수와 훈련하지 않는 변수를 관리한다.

In [17]:
w = tf.Variable([1, 2, 3], trainable=False)
print(w.trainable) # 훈련하지 않는 변수

False


In [18]:
# Variable 값은 .assign(), .assign_add() 메서드 등과 같은 연산을 실행하여 수정
print(w.assign([3, 1, 4], read_value=True))
w.assign_add([2, -1, 2], read_value=False)
print(w.value())

<tf.Variable 'UnreadVariable' shape=(3,) dtype=int32, numpy=array([3, 1, 4], dtype=int32)>
tf.Tensor([5 0 6], shape=(3,), dtype=int32)


In [19]:
# 고전적인 랜덤 초기화 방법
# 원하는 출력 텐서 크기를 지정하여 이 객체를 호출
tf.random.set_seed(1)
init = tf.keras.initializers.GlorotNormal()
tf.print(init(shape=(3,)))

[-0.722795904 1.01456821 0.251808226]


In [20]:
# 위 객체를 사용하여 2 * 3 크기의 Variable을 초기화
v = tf.Variable(init(shape=(2, 3)))
tf.print(v)

[[0.28982234 -0.782292783 -0.0453658961]
 [0.960991383 -0.120003454 0.708528221]]


In [21]:
class MyModule(tf.Module):
    def __init__(self):
        init = tf.keras.initializers.GlorotNormal()
        self.w1 = tf.Variable(init(shape=(2, 3)), trainable=True)
        self.w2 = tf.Variable(init(shape=(1, 2)), trainable=False)

m = MyModule()
print('모든 변수: ', [v.shape for v in m.variables])
print('훈련 가능한 변수: ', [v.shape for v in m.trainable_variables])

모든 변수:  [TensorShape([2, 3]), TensorShape([1, 2])]
훈련 가능한 변수:  [TensorShape([2, 3])]


In [22]:
# @tf.function : 트레이싱과 그래프 생성을 통해 Variable 객체를 사용
# 텐서플로는 데코레이터가 적용된 함수 안에서 Variable 객체를 만들지 못하게 하려고 에러 발생함.
@tf.function
def f(x):
    w = tf.Variable([1, 2, 3])
f([1])

ValueError: ignored

In [24]:
# 에러를 피하는 방법
# 데코레이터가 적용된 함수 밖에서 Variable 객체를 정의하고 함수에서 사용
w = tf.Variable(tf.random.uniform((3, 3)))
@tf.function
def compute_z(x):
    return tf.matmul(w, x)
x = tf.constant([[1], [2], [3]], dtype=tf.float32)
tf.print(compute_z(x))

[[1.20935762]
 [3.89828062]
 [1.65398622]]
